In [74]:
from os import listdir
import pandas as pd

for filename in listdir('output'):
    if '_grid_search' in filename:
        print(filename)
    if '_stacknet' in filename:
        print(filename)

xgb_ordinal2_grid_search.csv
lgb_ordinal2_grid_search.csv
catb_ordinal2_grid_search.csv
lgb_ordinal2_stacknet.csv
catb_ordinal2_stacknet.csv
xgb_ordinal2_stacknet.csv


In [63]:
gs_res = pd.read_csv('output/xgb_ordinal2_grid_search.csv', index_col='Unnamed: 0').sort_values(by=['val_auc'], ascending=False)
#0.829998
gs_res.head()

,alpha,best_round,booster,colsample_bylevel,colsample_bytree,cv,cv_nfold,cv_stratified,data_name,early_stopping_rounds,...,objective,pred_timespent,scale_pos_weight,seed,subsample,timestamp,train_all_auc,train_auc,tree_method,val_auc
PJFH,0,11,gbtree,0.675,0.7,True,2,False,ordinal2,1,...,binary:logistic,0:00:00,1,30286353,0.7,2018-07-29 16:34:30,0.726877,0.738481,hist,0.689073


In [47]:
sn_res = pd.read_csv('output/catb_ordinal2_stacknet.csv', index_col='Unnamed: 0').sort_values(by=['cv_val_auc'], ascending=False)
#0.829998
sn_res.head()

,cv_train_auc,cv_val_auc,data_name,depth,eval_metric,iterations,l2_leaf_reg,logging_level,oof_seed,random_seed,thread_count
aiHv,0.727298,0.700181,ordinal2,7,AUC,100,0.2,Verbose,16950940,16950940,15
OG7w,0.730607,0.699527,ordinal2,8,AUC,99,1.6,Verbose,6807070,6807070,15


In [9]:
gs_catb_res = pd.read_csv('output/catb_ordinal2_grid_search.csv', index_col='Unnamed: 0').sort_values(by=['val_AUC'], ascending=False)
gs_catb_res.head()

,best_round,cv,data_name,depth,eval_metric,gs_timespent,iterations,l2_leaf_reg,logging_level,pred_timespent,random_seed,thread_count,timestamp,train_AUC,train_all_AUC,val_AUC
X6uo,30,False,ordinal2,10,AUC,0:00:04,30,0.7,Verbose,0:00:06,25367694,15,2018-07-25 16:22:29,0.706230,0.737684,0.701732
UiBN,30,False,ordinal2,7,AUC,0:00:02,30,1.1,Verbose,0:00:02,27857633,15,2018-07-27 13:13:59,0.704351,0.714771,0.700455
Val5,30,False,ordinal2,8,AUC,0:00:03,30,0.7,Verbose,0:00:03,33892013,15,2018-07-25 16:07:32,0.707702,0.722036,0.699959
yp1r,30,False,ordinal2,9,AUC,0:00:02,30,2.2,Verbose,0:00:04,26076149,15,2018-07-25 16:07:25,0.704676,0.727607,0.698954


In [4]:
gs_lgb_res = pd.read_csv('output/lgb_ordinal2_grid_search.csv', index_col='Unnamed: 0').sort_values(by=['val_auc'], ascending=False)
#0.829998
gs_lgb_res.head()

,bagging_fraction,bagging_freq,bagging_seed,best_round,boosting,cv,data_name,early_stopping_round,feature_fraction,feature_fraction_seed,...,num_boost_round,num_leaves,num_threads,objective,pred_timespent,scale_pos_weight,timestamp,train_all_auc,train_auc,val_auc
DQ7E,0.9,1,24200148,30,gbdt,False,ordinal2,10,1.0,24200148,...,30,31,8,binary,0:00:00,1,2018-07-25 16:07:23,0.798216,0.803446,0.730565
wTAx,0.9,1,25274724,30,gbdt,False,ordinal2,10,1.0,25274724,...,30,15,8,binary,0:00:00,1,2018-07-25 16:07:24,0.765827,0.767328,0.727868


In [5]:
gs_nn_res = pd.read_csv('output/nn_ordinal2_grid_search.csv', index_col='Unnamed: 0').sort_values(by=['val_auc'], ascending=False)
gs_nn_res.head()

,batch_size,best_epoch,cat_emb_drop_rate,cat_emb_outdim,combined_layers_dense_units,combined_layers_drop_rate,data_name,ep_for_lr,gs_timespent,int_list,...,patience,pred_batch_size,pred_timespent,timestamp,train_all_auc,trn_acc,trn_loss,val_acc,val_auc,val_loss
RStN,256,2,0.2,30,"""[100, 50]""",0.1,ordinal2,2,0:00:11,"""['num_layers_dense_units', 'combined_layers_d...",...,1,5000,0:00:00,2018-07-25 16:07:41,-1,0.813647,0.438728,0.806933,0.732794,0.442593
TN9s,512,4,0.1,30,"""[100, 50]""",0.1,ordinal2,2,0:00:10,"""['num_layers_dense_units', 'combined_layers_d...",...,1,5000,0:00:00,2018-07-25 16:07:54,-1,0.816494,0.431491,0.807733,0.732308,0.442544


In [83]:
stacknet_res = pd.read_csv('output/oof/stacknet_report.csv').sort_values(by=['oof_cv_score'], ascending=False)
stacknet_res['num_model_data'] = stacknet_res['chosen model_data'].apply(lambda x: len(x.split('|')) if x is not np.nan else 0)
stacknet_res

,model_data,gs_val_auc,test_score,oof_cv_score,timestamp,seed,nfolds,chosen model_data,num_model_data
15,i5S4__LOGREG_layer2,NaN,0.716434,0.716992,2018-07-30 13:11:37,NaN,3,qZAR__LGB | mFju__CATB | CxxQ__XGB | bsFe__XGB...,12
9,N0sY__LOGREG_layer3,NaN,0.715586,0.716448,2018-07-30 11:54:42,NaN,3,qZAR__LGB | wlfr__LGB | D9UD__LGB | S2oN__LGB,4
17,kpHk__LOGREG_layer3,NaN,0.715586,0.716448,2018-07-30 13:11:38,NaN,3,qZAR__LGB | wlfr__LGB | D9UD__LGB | S2oN__LGB,4
16,idbI__LOGREG_layer3,NaN,0.715586,0.716448,2018-07-30 11:53:53,NaN,3,qZAR__LGB | wlfr__LGB | D9UD__LGB | S2oN__LGB,4
8,KBPg__LOGREG_layer2,NaN,0.715541,0.714690,2018-07-29 17:43:07,NaN,3,AYbJ__XGB | D9UD__LGB | UkLS__CATB | S2oN__LGB...,6
10,Pkwy__LOGREG_layer2,NaN,0.714894,0.714425,2018-07-30 13:11:37,NaN,3,wlfr__LGB | S2oN__LGB,2
22,vpeP__LOGREG_layer2,NaN,0.713532,0.712168,2018-07-29 17:36:42,NaN,3,qZAR__LGB | mFju__CATB | CxxQ__XGB | bsFe__XGB...,6
11,S2oN__LGB_ordinal2_layer1,NaN,0.712206,0.707990,2018-07-29 17:42:52,15404721.0,2,NaN,0
7,JBbu__LOGREG_layer2,NaN,0.711360,0.706068,2018-07-29 17:40:05,NaN,3,AYbJ__XGB | D9UD__LGB | UkLS__CATB,3
23,wlfr__LGB_ordinal2_layer1,0.702708,0.711788,0.705465,2018-07-29 17:35:37,31887101.0,2,NaN,0


In [ ]:
# 1. oof_cv_score vs. gs_val_auc
# for NN, oof_cv_score != gs_val_auc is normal, because gs_val_auc is get by train+val in grid search, while 
# oof_cv_score is got by nfolds cv
# for LGB, if gs_cv is true in config file, then oof_cv_score == gs_val_auc (almost). otherwise, based on the same
# reason as NN, oof_cv_score != gs_val_auc is normal 

# 2. test_score
# for NN: the one(not showing in log) is got by train+val and load_weights when doing prediction after grid search
# (when do_preds is true in config file), and test_score is got during oof where all trainig data are used. It's 
# normal that they are not equal
# for LGB, since model is retrain using all training data when doing prediction after grid search, the one is not in 
# log == test_score

In [77]:
import os
import sys
#nb_dir = os.path.split(os.getcwd())[0]
#if nb_dir not in sys.path:
#    sys.path.append(nb_dir)
    
autolib_dir = '/home/kai/data/shiyi/AlphaBoosting/automl'
if autolib_dir not in sys.path:
    sys.path.append(autolib_dir)

print(sys.path)

%load_ext autoreload
%autoreload 2

['', '/home/kai/anaconda3/envs/tf_gpu/lib/python35.zip', '/home/kai/anaconda3/envs/tf_gpu/lib/python3.5', '/home/kai/anaconda3/envs/tf_gpu/lib/python3.5/plat-linux', '/home/kai/anaconda3/envs/tf_gpu/lib/python3.5/lib-dynload', '/home/kai/.local/lib/python3.5/site-packages', '/home/kai/anaconda3/envs/tf_gpu/lib/python3.5/site-packages', '/home/kai/anaconda3/envs/tf_gpu/lib/python3.5/site-packages/IPython/extensions', '/home/kai/.ipython', '/home/kai/data/shiyi/AlphaBoosting/automl']


In [78]:
from automl_libs import BaseLayerResultsRepo

/home/kai/anaconda3/envs/tf_gpu/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [79]:
repo = BaseLayerResultsRepo(['label'], 'output/oof/', load_from_file=True)

In [80]:
for m,s in repo.show_scores():
    print(s,m)

0.7146899471276421 KBPg__LOGREG_layer2
0.7121681805081544 vpeP__LOGREG_layer2
0.7079895915409449 S2oN__LGB_ordinal2_layer1
0.706068486192415 JBbu__LOGREG_layer2
0.7054650156014354 wlfr__LGB_ordinal2_layer1
0.7042211871721642 qZAR__LGB_ordinal2_layer1
0.7027516979235752 D9UD__LGB_ordinal2_layer1
0.69047150751821 qcjp__XGB_ordinal2_layer1
0.6900295449740382 mFju__CATB_ordinal2_layer1
0.6893625821588822 BFdf__NN_layer2
0.6892801409517826 UKDO__CATB_ordinal2_layer1
0.688897253712206 olYz__CATB_ordinal2_layer1
0.6886861010809897 UkLS__CATB_ordinal2_layer1
0.6839137269122242 AYbJ__XGB_ordinal2_layer1
0.6810965121440378 D8a2__NN_layer2
0.6568122644120544 bsFe__XGB_ordinal2_layer1
0.6374596227893377 CxxQ__XGB_ordinal2_layer1
0.5623907319752555 76iL__NN_layer2


In [ ]:
gs_nn_res.head(0).T.to_dict()

In [ ]:
from sklearn.svm import SVC

In [ ]:
from sklearn.svm import LinearSVC
from sklearn.calibration import CalibratedClassifierCV

In [ ]:
from sklearn.ensemble import BaggingClassifier

In [ ]:
lsvc = CalibratedClassifierCV(LinearSVC())

In [ ]:
lsvc.fit(np.array([0,1,2,3,4,5]).reshape(-1,1), [1,0,0,1,1,0])

In [ ]:
lsvc.predict_proba(2.22)

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
svc = SVC(kernel='linear', probability=True)#, class_weight='auto')


In [ ]:
svc.fit(np.array([1,2,3,4,5]).reshape(-1,1), [0,0,1,1,0])

In [ ]:
svc.predict_proba(2.22)

In [ ]:
from sklearn.multiclass import OneVsRestClassifier

In [ ]:
clf = OneVsRestClassifier(BaggingClassifier(svc, max_samples=1.0 / 5, n_estimators=5))
clf.fit(np.array([1,2,3,4,5]).reshape(-1,1), [0,0,1,1,0])

In [ ]:
clf.predict_proba(2.22)